In [1]:
## Using CVXPY to solve a multi-scenario optimization problem
## Dual-Optimization setup to minimize residual load realtime

import numpy as np
import pandas as pd
import cvxpy as cp

linecut = 96
scenario1 = pd.read_csv('data/Scenario1.csv').iloc[:linecut]
scenario2 = pd.read_csv('data/Scenario2.csv').iloc[:linecut]
scenario3 = pd.read_csv('data/Scenario3.csv').iloc[:linecut]
scenario4 = pd.read_csv('data/Scenario4.csv').iloc[:linecut]
scenario5 = pd.read_csv('data/Scenario5.csv').iloc[:linecut]
scenarios_data = [scenario1, scenario2, scenario3, scenario4, scenario5]

# Fixed cost parameters
wind_cost_per_mw = 250
base_load_cost_per_mw = 360
peak_load_cost_per_mw = 200
load_following_option_fee = 50
load_following_exercise_fee = 18
service_fee = 5

max_wind_capacity = 20
max_generator_capacity = 10
solar_capacity = 1

wind_capacity = cp.Variable()
baseload_capacity = cp.Variable() # [MW]
peak_capacity = cp.Variable() # [MW]
load_following_capacity = cp.Variable()

wind_capacity.value = 10  # Example initial guess for wind capacity
baseload_capacity.value = 5  # Example initial guess for baseload capacity
peak_capacity.value = 5  # Example initial guess for peak capacity
load_following_capacity.value = 2  # Example initial guess for load-following capacity

x = [wind_capacity, baseload_capacity, peak_capacity, load_following_capacity]
numerical_x = [var.value for var in x]

constraints = [
    wind_capacity >= 0,
    baseload_capacity >= 0,
    peak_capacity >= 0,
    load_following_capacity >= 0,
    wind_capacity <= max_wind_capacity,
    baseload_capacity + peak_capacity + load_following_capacity <= max_generator_capacity
]


# Scenario cost with real-time load-following optimization
from scipy.optimize import minimize
def scenario_cost(x, demand, real_time_price, wind_factor, solar_factor):
    # Fixed cost regardless of real-time events
    fixed_cost = (x[0] * wind_cost_per_mw +
                  x[1] * base_load_cost_per_mw +
                  x[2] * peak_load_cost_per_mw +
                  x[3] * load_following_option_fee)

    demand_mw = demand.values / 1000
    real_time_price = real_time_price.values
    solar_gen = solar_capacity * solar_factor.values
    wind_gen = x[0] * wind_factor
    base_gen = x[1]
    peak_gen = x[2]

    # Initialize real-time cost
    real_time_cost = 0

    # Load-following optimization for each time step
    for t in range(len(demand_mw)):
        # Calculate unmet demand
        unmet_demand_t = cp.Parameter(nonneg=True)
        if 8 <= float(t) * 4 / 60 <= 16:  # During peak hours
            unmet_demand_t.value = max(demand_mw[t] - (solar_gen[t] + wind_gen[t] + base_gen + peak_gen), 0)
        else:  # Non-peak hours
            unmet_demand_t = max(demand_mw[t] - (solar_gen[t] + wind_gen[t] + base_gen), 0)

        # Define the objective function for scipy.optimize
        def lower_objective(z):
            load_follow_gen_t, spot_load_t = z
            # Cost function for realtime unmet demand optimization
            return load_follow_gen_t * load_following_exercise_fee + spot_load_t * (real_time_price[t] + service_fee)

        # Define the constraints
        constraints = [
            {'type': 'ineq', 'fun': lambda z: z[0]},  # load_follow_gen_t >= 0
            {'type': 'ineq', 'fun': lambda z: x[3] - z[0]},  # load_follow_gen_t <= load_following_capacity
            {'type': 'ineq', 'fun': lambda z: z[1]},  # spot_load_t >= 0
            {'type': 'ineq', 'fun': lambda z: z[0] + z[1] - unmet_demand_t}  # load_follow_gen_t + spot_load_t >= unmet_demand_t
        ]

        # Initial guess for [load_follow_gen_t, spot_load_t]
        # initial_guess = cp.Parameter(2, nonneg=True)
        initial_guess = [0, unmet_demand_t] # Start with no load-following generation, spot covers all

        # Solve the sub-problem using scipy.optimize
        result = minimize(lower_objective, initial_guess, constraints=constraints, bounds=[(0, unmet_demand_t), (0, unmet_demand_t)])
        
        if result.success:
            print(result.fun)
            real_time_cost += result.fun  # Add the minimized cost for this time step
        else:
            print(f"Optimization failed at time step {t} with unmet demand {unmet_demand_t}")

    # Total scenario cost
    return fixed_cost + real_time_cost


# Aggregate cost function across all scenarios
def aggregate_cost(x):
    total_cost = 0
    for scenario in scenarios_data:
        total_cost += scenario_cost(
            x,
            demand=scenario['Demand [kW]'],
            real_time_price=scenario['Real Time Price [$/MWh]'],
            wind_factor=scenario['Wind Power Factor [p.u.]'],
            solar_factor=scenario['Solar Power Factor [p.u.]']
        )
    return total_cost / len(scenarios_data)

# Objective: Minimize the average cost across all scenarios
objective = cp.Minimize(aggregate_cost(numerical_x))

# Define and solve the problem using the Gurobi solver
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI, verbose='true')

# Output results
if problem.status == cp.OPTIMAL:
    output = {
        "Minimum expected cost": problem.value,
        "Wind capacity (MW)": wind_capacity.value,
        "Baseload capacity (MW)": baseload_capacity.value,
        "Peak load capacity (MW)": peak_capacity.value,
        "Load following capacity (MW)": load_following_capacity.value
    }
else:
    output = {"error": problem.status}

output


33.76224000020471
32.725439999973624
32.932799999939874
34.73280000000883
33.85152000001777
31.844160000018533
29.42207999997315
29.111039999947963
30.755520000246268
37.901633600123496
39.795635199829654
39.67881280025483
38.778655999912836
29.363526400000005
29.342217600000005
24.7756416
29.525376
29.623126399999997
30.099708800000005
41.15388159987974
43.90051199994356
47.15586559997679
50.786639999843146
53.24054080019362
57.44985920019015
58.2009311998263
60.392262399774836
60.649423999805286
46.59969599999999
57.696288
62.41898239999998
58.253852800000004
29.91761600000001
23.8582656
28.849128000000004
28.466207999999995
31.937628000000004
82.45672959998541
64.2526896
64.42649999999998
74.93800800004553
68.67436800000003
70.93751999999999
75.15754000021585
95.21055280043367
83.53899840019321
83.88272880026369
82.60123999998426
83.27935200005223
96.25457680009794
97.50205599998614
92.57626799993417
88.7262287998965
102.80308319958006
93.37805439991075
95.77246080011992
65.50326399

TypeError: an integer is required